# Conflate travel demand model attributes.

### Prepare the data for processing.

Workspace, spatial reference. Point it to your local file geodatabase. 

In [ ]:
import arcpy
arcpy.env.overwriteOutput = True

# authentication to access secured data
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
gis = GIS('pro')

db = arcpy.env.workspace = r".\data\tdm_conflation_0324.gdb"
arcpy.Describe(db)

target_sr = arcpy.SpatialReference(2223)

Source data URL, key fields.

In [ ]:
route_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Routes/FeatureServer/92'
# This is the sun cloud routes layer where the road representation type was indentified by Mark E. 
route_by_type = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/ArcGIS/rest/services/Segmented_Routes_by_Line_Representation_Type/FeatureServer/0'
key_fields = [
    'ABLANES2019',
    'BALANES2019',
    'ABLANES2050',
    'BALANES2050',
    'ABDAYVOL2019',
    'BADAYVOL2019',
    'TOTDAYVOL2019',
    'ABDAYVOL2050',
    'BADAYVOL2050',
    'TOTDAYVOL2050',
    'ABCAPACITY2019',
    'BACAPACITY2019',
    'ABCAPACITY2050',
    'BACAPACITY2050',]

Custom functions. 

In [ ]:
# fill the shor segments with the boundary touching segment that has the same route ID
def fill_short(distance, query):
    fc = 'destination'
    arcpy.Delete_management('short_lyr')
    arcpy.Delete_management('full_lyr')

    arcpy.management.MakeFeatureLayer(fc, 'short_lyr', 'Shape_Length <{} and {sun_cloud_id is null}'.format(distance))
    print(arcpy.GetCount_management('short_lyr')[0])

    arcpy.management.MakeFeatureLayer(fc, 'full_lyr', 'sun_cloud_id is not null')


    fields = ['SHAPE@', 'OID@', 'route_id', 'sun_cloud_id']

    with arcpy.da.UpdateCursor('short_lyr', fields) as cursor:
        for row in cursor:
            geom = row[0]
            # select full layer that touches boundary of short layer.
            selected = arcpy.SelectLayerByLocation_management('full_lyr', 'BOUNDARY_TOUCHES', geom, '', 'NEW_SELECTION')
            sql="route_id = '{}'".format(row[2])
            # print(sql)
            # subselect where route_id matches. 
            sub_selection = arcpy.SelectLayerByAttribute_management(selected, 'SUBSET_SELECTION', sql)

            sid = [r[0] for r in arcpy.da.SearchCursor(sub_selection, ['sun_cloud_id'])]
            # notes = [r[0] for r in arcpy.da.SearchCursor(sub_selection, ['notes'])]
            # print(sid)
            # if there are multiple segments that meet the condition, just grab the first one.
            if len(sid)>0:
                print(sid)
                row[3]=sid[0]
            
            cursor.updateRow(row)

# get unique field values from a table
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

# save hosted feature layer to db
def save_fl(db, url, outname):
    fl = FeatureLayer(url)
    featureset = fl.query()
    featureset.save(db, outname)

# project in_data to match the target spatial reference
def project(in_data, target_sr, out_name):


    in_sr = arcpy.Describe(in_data).spatialReference

    tr = arcpy.ListTransformations (in_sr, target_sr)
    datum_conversion = ''
    if(len(tr)>0):
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]

    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)

# standardize road name
import re
def strip_stop_words(x):
# remove leading zeros
    x =  x.lower()

    stop_words = ['avenue', 'ave', 
            'boulevard', 'blvd', 
            'drive', 'dr', 
            'freeway', 'frwy', 'fwy',
            'lane', 'ln',
            'parkway', 'pkwy',  
            'road', 'rd', 
            'route', 'rte',
            'street', 'st', 
            'trail', 'tr',
            'way',

            'railroad', 'chn', 'drainage', 'lake', 'siphons', 'track', 'bnsf', 
            'place', 'pl', 
            'l', 'n', 's', 'e', 'w', 'irr', 'i', 'us', 'to', '-', 'from',
            'sl', 'sr', 'loop', 'lp', 'frtg', 
            'sb', 'wb', 'eb', 'nb', 'direct', 'hov', 'ramp', 'n-w', 'w-s', 'n-e']
            
    x = re.split(r"\'|\s|;|,|/|-|\(|\)", x)
    # print(x)
    x = [y for y in x if y.lower() not in stop_words]
    # clean_list = [n for n in x if n.strip()]
    remove_zero = [item.lstrip('0') for item in x]
    clean= [n for n in remove_zero if n.strip()]
    return(clean)

# use this to check joined field names
def joinCheck(lyr):
  fList = arcpy.Describe(lyr).fields
  for f in fList:
    print (f.name)

# returns total count of a feature class, feature layer
def get_count(fc):
    cnt = int(arcpy.GetCount_management(fc)[0])
    print(cnt)
    return cnt

Save the routes layer locally and project if needed.

In [ ]:

fc = 'route_by_representation'
if arcpy.Exists(fc)==False:
    save_fl(db, route_by_type, fc)

### Prepare the source layer.

Project the data.

In [ ]:
source_fc = 'SunCloudBaseandFutureVolumesCapacityV3'
sr = arcpy.Describe(source_fc).spatialReference
if target_sr != sr:
    print('projecting...')
    project(source_fc, target_sr, 'tdm_project')

Delete the connectors and ramps (they are not a part of the sun cloud routes).

In [ ]:
fc = 'tdm_project'
print(arcpy.GetCount_management(fc).getOutput(0))
sql = "FUNCLASSDESC IN ('Connect HOV & GP lanes', 'Ramp')"

selection = arcpy.management.SelectLayerByAttribute(fc, 'NEW_SELECTION', sql)

cnt = int(arcpy.GetCount_management(selection).getOutput(0))
print(cnt)
if cnt>0 :
    print('deleting...')
    arcpy.DeleteFeatures_management(selection)

print(arcpy.GetCount_management(fc).getOutput(0))


### Split the destination to match the source segments.
Over splitting is acceptable as we will eventually unsplit the final result layer.

Project the 'route by representation' layer.

In [ ]:
#project the route by type
fc = 'route_by_representation'
project(fc, target_sr, '{0}_project'.format(fc))

In [ ]:
in_fc1 = 'sun_cloud_segmented_route'
# feature class that has road representation type value.
in_fc2 = '_destination'
out_intersect = 'temp_intersect'
arcpy.analysis.Intersect(
    in_features="{0};{1}".format(in_fc1, in_fc2),
    out_feature_class=out_intersect,
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)

joined = arcpy.management.AddJoin(
    in_layer_or_view=in_fc1,
    in_field="OBJECTID",
    join_table=out_intersect,
    join_field="FID_sun_cloud_segmented_route",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)

print(arcpy.GetCount_management(joined)[0])
joinCheck(joined)


In [ ]:
# transfer over the "type(road representation type)"
arcpy.management.CalculateField(
    in_table=joined,
    field="{}.type".format(in_fc1),
    expression="!{}.type_1!".format(out_intersect),
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Create end points from the source layer. We'll use this to split the destination.

In [ ]:
fc = 'tdm_project'
end_pt= "tdm_end_pts"

arcpy.management.FeatureVerticesToPoints(fc, end_pt, "BOTH_ENDS")

Snap end points to the routes & split (oversplitting is fine, we'll run unsplit at the end of the process.)

In [ ]:
target = 'sun_cloud_segmented_route'
arcpy.edit.Snap('tdm_end_pts',[[target, "EDGE", "300 Feet"]])

In [ ]:
target = 'sun_cloud_segmented_route'
end_pt = 'tdm_end_pts'
search_distance = '300 Feet'
arcpy.management.SplitLineAtPoint(target, end_pt, 'destination', search_distance)

Separate out the HOV lane. We'll conflate this separately and add the volume/capacity to correndponding freeway.

In [ ]:
fc = 'tdm_project'
sql = "FUNCLASSDESC = 'Freeway HOV Lane'"

arcpy.conversion.FeatureClassToFeatureClass(fc, db, 'tdm_hov', sql)

selection = arcpy.management.SelectLayerByAttribute(fc, 'NEW_SELECTION', sql)

cnt = int(arcpy.GetCount_management(selection).getOutput(0))

print(cnt)
if cnt>0 :
    arcpy.DeleteFeatures_management(selection)
    print(arcpy.GetCount_management(fc).getOutput(0))

### Prepare the data for conflation.

Standardize the road name value.

In [ ]:
#  clean up
fc = 'destination'
arcpy.DeleteField_management(fc, ['ORIG_FID', 'ORIG_SEQ'])

custom_code = """
import re
def strip_stop_words(x):

    x =  x.lower()
    stop_words = ['avenue', 'ave', 
            'boulevard', 'blvd', 
            'drive', 'dr', 
            'freeway', 'frwy', 'fwy',
            'lane', 'ln',
            'parkway', 'pkwy',  
            'road', 'rd', 
            'route', 'rte',
            'street', 'st', 
            'trail', 'tr',
            'way', 'hwy', 'highway',

            'railroad', 'chn', 'drainage', 'lake', 'siphons', 'track', 'bnsf', 
            'place', 'pl', 
            'l', 'n', 's', 'e', 'w', 'irr', 'i', 'us', 'to', '-', 'from',
            'sl', 'sr', 'loop', 'lp', 'frtg', 
            'sb', 'wb', 'eb', 'nb', 'direct', 'hov', 'ramp', 'n-w', 'w-s', 'n-e']
            
    x = re.split(r"\'|\s|;|,|/|-|\(|\)", x)

    x = [y for y in x if y.lower() not in stop_words]
    # clean_list = [n for n in x if n.strip()]
    remove_zero = [item.lstrip('0') for item in x]
    clean= [n for n in remove_zero if n.strip()]
    return ' '.join(clean)
    """

arcpy.management.CalculateField(
    in_table=fc,
    field="clean_name",
    expression="strip_stop_words(!route_id!)",
    expression_type="PYTHON3",
    code_block= custom_code,
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="tdm_project",
    field="clean_name",
    expression="strip_stop_words(!ST_NAME!)",
    expression_type="PYTHON3",
    code_block= custom_code,
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Add the key fields to destination.

In [ ]:
fc = 'destination'
fields = [
['sun_cloud_id', 'TEXT'],   
['ABLANES2019', 'SHORT'],
['BALANES2019','SHORT'],
['ABLANES2050','SHORT'],
['BALANES2050','SHORT'],
['ABDAYVOL2019','DOUBLE'],
['BADAYVOL2019','DOUBLE'],
['TOTDAYVOL2019','DOUBLE'],
['ABDAYVOL2050', 'DOUBLE'],
['BADAYVOL2050','DOUBLE'],
['TOTDAYVOL2050','DOUBLE'],
['ABCAPACITY2019','DOUBLE'],
['BACAPACITY2019','DOUBLE'],
['ABCAPACITY2050','DOUBLE'],
['BACAPACITY2050','DOUBLE'],
['notes', 'TEXT'],
['conflation_type','TEXT']
]


arcpy.management.AddFields(
    fc,fields)
featureclass = fc
field_names = [f.name for f in arcpy.ListFields(featureclass)]
field_names


### Match the destination to source through buffer, join, and validation.

- For each segment where sun cloud id is null, create a buffer.
- Intersect the buffer to TDM source layer. 
  - Validate road name and functional class.
  - Delete out the bad matches. 
  - Find the longest match from the multiple valid matches. 
- Join the routes layer to the longest valid match and attribute the Sun Cloud ID. 

Review the result and repeat the process while increasing the buffer distance. 

In [ ]:
arcpy.AddField_management('tdm_project', 'type','TEXT')
custom_code = """

def get_type(f):
    if f in [-1, 1]: 
        return 'dual_line'
    elif f in [0]:
        return 'single_line'
"""


arcpy.management.CalculateField(
    in_table="tdm_project",
    field="type",
    expression="get_type(!DIR!)",
    expression_type="PYTHON3",
    code_block= custom_code,
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Transfer the sun cloud ID. 

In [ ]:
! pip install fuzzywuzzy

In [ ]:
def tdm_conflation(distance, in_fc):
    sql_query = "sun_cloud_id is null"
    arcpy.Delete_management('destination_lyr')

    arcpy.management.SelectLayerByAttribute( in_fc, 'CLEAR_SELECTION') 
    arcpy.management.MakeFeatureLayer(in_features=in_fc, out_layer='destination_lyr', where_clause=sql_query)
    
    # Create buffer around the routes with no SC ID. 
    buffer_table = "destination_Buffer_{}".format(distance)
    arcpy.analysis.Buffer(
        in_features='destination_lyr',
        out_feature_class="destination_Buffer_{}".format(distance),
        buffer_distance_or_field="{} FeetInt".format(distance),
        line_side="FULL",
        line_end_type="FLAT",
        dissolve_option="NONE",
        dissolve_field=None,
        method="PLANAR"
    )

    # create a intersect table
    intersect = "{}_intersect".format(buffer_table)
    arcpy.analysis.PairwiseIntersect(
        in_features="{};tdm_project".format(buffer_table),
        out_feature_class=intersect,
        join_attributes="ALL",
        cluster_tolerance=None,
        output_type="INPUT"
    )

    custom_code = """
from fuzzywuzzy import fuzz

def func_match(functional_class, FUNCLASSCODE, type, type_1, desc):

    if (desc ==  'Freeway' and functional_class in [1,2,3]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc ==  'Expressway' and functional_class in [2,3]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc == '6 Legged Arterial' and functional_class in [2, 4]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc in ['Arterial']  and functional_class in [2,3,4,5, 6]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc in ['Collector/Frontage Rd', 'Major Collector', 'Minor Arterial']  and functional_class in [3,4,5, 6]):
        result = '{0} to {1}'.format(type_1, type)

    elif (desc in ['Collector/Frontage Rd']  and functional_class in [3,4,5,6,7]):
        result = '{0} to {1}'.format(type_1, type)

    elif (desc == 'CD Road' and functional_class in [2,3,4,5,6,7]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc == 'Collector' and functional_class in [4,5]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc ==  'Major Arterial' and functional_class in [3, 4]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc == 'Principal Arterial' and functional_class in [3]):
        result = '{0} to {1}'.format(type_1, type)
    elif (desc == 'Local' and functional_class in [7]):
        result = '{0} to {1}'.format(type_1, type)
    else:
        result = 'fc mismatch'
    return result
    
def validate_join_strict (clean_name, clean_name_1, functional_class, FUNCLASSCODE, type, type_1, desc):
    # print(clean_name, clean_name_1, functional_class, FUNCLASSCODE, type, type_1, desc)
    result = None
    # if name value exists
    if (clean_name and clean_name_1):
        # print('{0}, {1}'.format(clean_name, clean_name_1))
        if (clean_name_1 in clean_name) or (clean_name in clean_name_1):
            # print('name match')
            result = '{0} to {1}'.format(type_1, type)
        else:
            name_match_ratio = fuzz.partial_ratio(clean_name, clean_name_1)
            # print(name_match_ratio)
            if name_match_ratio > 80:
                result = '{0} to {1}'.format(type_1, type)
            else:
                result = func_match(functional_class, FUNCLASSCODE, type, type_1, desc)

    else:
        result = func_match(functional_class, FUNCLASSCODE, type, type_1, desc)

    return result
    """


    # compare standardized name and functional class. 
    arcpy.management.CalculateField(
        in_table=intersect,
        field="notes",
        expression="validate_join_strict(!clean_name!, !clean_name_1!, !functional_class!, !FUNCLASSCODE!, !type!, !type_1!,!FUNCLASSDESC!)",
        expression_type="PYTHON3",
        code_block=custom_code,
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    
    # export the full table before row deletion
    arcpy.ExportFeatures_conversion(intersect, '{}_backup'.format(intersect))

    sql_query = "notes is null or notes IN ('name mismatch', 'missing name', 'fc mismatch')"

    # delete the bad matches from the intersect table
    selected = arcpy.SelectLayerByAttribute_management(intersect, 'NEW_SELECTION', sql_query)
    cnt = int(arcpy.GetCount_management(selected)[0])
    if(cnt>0):
        arcpy.DeleteFeatures_management(selected)
    # get all unique orig_fids
    org_fids = [row[0] for row in arcpy.da.SearchCursor(
                in_table=intersect, 
                field_names='ORIG_FID', 
                sql_clause= ('DISTINCT ORIG_FID', 'ORDER BY ORIG_FID ASC'))]
     
    # For Each OID, leave the longest segment and delete the rest
    fields = ['Shape_Length']
    for oid in org_fids:
        print('Processing oid = {}'.format(oid))
        _query = 'ORIG_FID={}'.format(oid)
        arcpy.MakeFeatureLayer_management(intersect, 'intersect_lyr')
        sel = arcpy.SelectLayerByAttribute_management('intersect_lyr', 'NEW_SELECTION', _query)

        # get_count(sel)

        length = [row[0] for row in arcpy.da.SearchCursor(
                in_table=sel, 
                field_names='Shape_Length')]

        max_len = max(length)
        _query = 'ORIG_FID = {} and Shape_Length <{}'.format(oid, max_len)
        sub_sel = arcpy.SelectLayerByAttribute_management(sel, 'SUBSET_SELECTION', _query)
        if int(arcpy.GetCount_management(sub_sel)[0])>0:

            arcpy.DeleteFeatures_management(sub_sel)

    # join it back to the routes table and transfer the sc id. 
    arcpy.Delete_management('null_layer')
    arcpy.MakeFeatureLayer_management(in_fc, 'null_layer', 'sun_cloud_id is null')
    arcpy.Delete_management('intersect_lyr')   
    arcpy.MakeFeatureLayer_management(intersect, 'intersect_lyr')


    joined = arcpy.management.AddJoin(
        in_layer_or_view="null_layer",
        in_field="OBJECTID",
        join_table='intersect_lyr',
        join_field="ORIG_FID",
        join_type="",
        index_join_fields="INDEX_JOIN_FIELDS"
        )

    arcpy.management.CalculateField(

        in_table=joined,
        field="{}.notes".format(in_fc),
        expression="""!{}.notes!""".format(intersect),
        expression_type="PYTHON3",
        code_block="",
        field_type="TEXT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

    arcpy.management.CalculateField(
        in_table=joined,
        field="{}.sun_cloud_id".format(in_fc),
        expression="""!{}.SUNCLOUDID!""".format(intersect),
        expression_type="PYTHON3",
        code_block="",
        field_type="TEXT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )
    # garbage collection
    arcpy.Delete_management(joined)


In [ ]:
distances = [20, 50, 100, 150, 200, 250 300]
for d in distances:
    print(d)
    tdm_conflation(d, 'destination')

Fill the short segments using the neighboring segment's information (if length is less than n feet and route id is the same, attribute.)

In [ ]:
fill_short(50)

**In the map, visually examine and mark the segments that do not have TDM matches. ("no_source")**

- There are functional class mismatches due to errornous value in the source data. They need to be manually attributed. 

Determine the conflation method based on the road representation type. 

In [ ]:
fc = 'destination'

joined = arcpy.management.AddJoin(
    in_layer_or_view=fc,
    in_field="sun_cloud_id",
    join_table='tdm_project',
    join_field="SUNCLOUDID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
    )
get_count(joined)
joinCheck(joined)

In [ ]:

arcpy.management.CalculateField(
    in_table=joined,
    field="{}.conflation_type".format(fc),
    expression='"{0} to {1}".format(!tdm_project.type!, !destination.type!)',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.Delete_management(joined)

### Merge attributes by road representation type.

#### Simple cases.

In [ ]:
fc = 'destination'
# sql="conflation_type in ('dual_line to dual_line', 'single_line to single_line')"
sql="conflation_type in ('simple reprocess')"

arcpy.MakeFeatureLayer_management(fc, 'simple_lyr', sql)
print(arcpy.GetCount_management('simple_lyr')[0])

In [ ]:
joined = arcpy.management.AddJoin(
    in_layer_or_view='simple_lyr',
    in_field="sun_cloud_id",
    join_table="tdm_project",
    join_field="SUNCLOUDID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)

In [ ]:
get_count(joined)
joinCheck(joined)

In [ ]:
# transfer over all the key field values
for k in key_fields:
    print(k)
    arcpy.management.CalculateField(
    in_table=joined,
    field="{0}.{1}".format(fc, k),
    expression="!tdm_project.{}!".format(k),
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

#### Single line to dual line. 

Attribute AB_ value to a direction and the BA_value to the opposite direction. 

In [ ]:
fc = 'destination'
sql = "conflation_type = 'single_line to dual_line'"
selected = arcpy.SelectLayerByAttribute_management(fc, 'NEW_SELECTION', sql)
get_count(selected)

In [ ]:
ref_fields = ['SUNCLOUDID']
a_fields = ['ABLANES2019', 'ABLANES2050', 'ABDAYVOL2019', 'ABDAYVOL2050', 'ABCAPACITY2019','ABCAPACITY2050']
b_fields = ['BALANES2019', 'BALANES2050', 'BADAYVOL2019', 'BADAYVOL2050', 'BACAPACITY2019', 'BACAPACITY2050']
ref_fields = ref_fields + a_fields + b_fields

sc_ids = unique_values(selected, 'sun_cloud_id')
total = len(sc_ids)
i=0
for sid in sc_ids:
    # if sid != 'MAG_33518':
    if 1==1:
        i=i+1
        print('{0}/{1}'.format(i, total))

        q = "sun_cloud_id = '{}'".format(sid)
        print(q)

        
        clean_names = [row[0] for row in arcpy.da.SearchCursor(
            in_table=selected, 
            field_names='clean_name', 
            where_clause=q,
            sql_clause= ('DISTINCT', 'ORDER BY route_id'))]

        print(clean_names)
        if len(clean_names)>1:
            for name in clean_names:
                q = "sun_cloud_id = '{0}' and clean_name = '{1}'".format(sid, name)
                
                route_ids = [row[0] for row in arcpy.da.SearchCursor(
                    in_table=selected, 
                    field_names='route_id', 
                    where_clause=q,
                    sql_clause= ('DISTINCT', 'ORDER BY route_id'))]
                print(route_ids)


                route_ids = [row[0] for row in arcpy.da.SearchCursor(
                    in_table=selected, 
                    field_names='route_id', 
                    where_clause=q,
                    sql_clause= ('DISTINCT', 'ORDER BY route_id'))]
                # print(route_ids)
                route_ids = list(set(route_ids))
                print(route_ids)
                length = len(route_ids)
                print(length)
                # if there are two distinct route IDs, split and assign values to each direction
                if length==2:
                    # get the key field values 
                    value = [row for row in arcpy.da.SearchCursor('tdm_project', ref_fields, "SUNCLOUDID ='{}'".format(sid))][0]
                    # print(value)

                    a_query = "sun_cloud_id = '{0}' and route_id='{1}'".format(sid, route_ids[0])
                    with arcpy.da.UpdateCursor (fc, a_fields, a_query) as cursor:
                        for row in cursor:
                            row[0] = int(value[1] or 0)
                            row[1] = int(value[2] or 0)
                            row[2] = int(value[3] or 0)
                            row[3] = int(value[4] or 0)
                            row[4] = int(value[5] or 0)
                            row[5] = int(value[6] or 0)
                            print('updated a fields')
                            cursor.updateRow(row)

                    
                    b_query = "sun_cloud_id = '{0}' and route_id='{1}'".format(sid, route_ids[1])
                    with arcpy.da.UpdateCursor (fc, b_fields, b_query) as Bcursor:
                        for Brow in Bcursor:
                            Brow[0] = int(value[7] or 0)
                            Brow[1] = int(value[8] or 0)
                            Brow[2] = int(value[9] or 0)
                            Brow[3] = int(value[10] or 0)
                            Brow[4] = int(value[11] or 0)
                            Brow[5] = int(value[12] or 0)
                            print('updated b fields')
                            Bcursor.updateRow(Brow)
                if length==1:
                    query = "sun_cloud_id = '{0}'".format(sid, route_ids[0])
                    with arcpy.da.UpdateCursor (fc, 'conflation_type', query) as cursor:
                        for row in cursor:
                            row[0]='simple'
                            cursor.updateRow(row)
        else:
                route_ids = [row[0] for row in arcpy.da.SearchCursor(
                    in_table=selected, 
                    field_names='route_id', 
                    where_clause=q,
                    sql_clause= ('DISTINCT', 'ORDER BY route_id'))]
                print(route_ids)


                route_ids = [row[0] for row in arcpy.da.SearchCursor(
                    in_table=selected, 
                    field_names='route_id', 
                    where_clause=q,
                    sql_clause= ('DISTINCT', 'ORDER BY route_id'))]
                # print(route_ids)
                route_ids = list(set(route_ids))
                print(route_ids)
                length = len(route_ids)
                print(length)
                # if there are two distinct route IDs, split and assign values to each direction
                if length==2:
                    # get the key field values 
                    value = [row for row in arcpy.da.SearchCursor('tdm_project', ref_fields, "SUNCLOUDID ='{}'".format(sid))][0]
                    # print(value)

                    a_query = "sun_cloud_id = '{0}' and route_id='{1}'".format(sid, route_ids[0])
                    with arcpy.da.UpdateCursor (fc, a_fields, a_query) as cursor:
                        for row in cursor:
                            row[0] = int(value[1] or 0)
                            row[1] = int(value[2] or 0)
                            row[2] = int(value[3] or 0)
                            row[3] = int(value[4] or 0)
                            row[4] = int(value[5] or 0)
                            row[5] = int(value[6] or 0)
                            print('updated a fields')
                            cursor.updateRow(row)

                    
                    b_query = "sun_cloud_id = '{0}' and route_id='{1}'".format(sid, route_ids[1])
                    with arcpy.da.UpdateCursor (fc, b_fields, b_query) as Bcursor:
                        for Brow in Bcursor:
                            Brow[0] = int(value[7] or 0)
                            Brow[1] = int(value[8] or 0)
                            Brow[2] = int(value[9] or 0)
                            Brow[3] = int(value[10] or 0)
                            Brow[4] = int(value[11] or 0)
                            Brow[5] = int(value[12] or 0)
                            print('updated b fields')
                            Bcursor.updateRow(Brow)
                if length==1:
                    query = "sun_cloud_id = '{0}'".format(sid, route_ids[0])
                    with arcpy.da.UpdateCursor (fc, 'conflation_type', query) as cursor:
                        for row in cursor:
                            row[0]='simple'
                            cursor.updateRow(row)
            



#### Dual line to single line. 

Get two SUNCLOUD ID for dual to single cases. 

In [ ]:
arcpy.env.overwriteOutput = True
# Create a buffer
sql = "conflation_type = 'dual_line to single_line'"
fc = "destination"
# sql = "ABDAYVOL2019 IS NULL And BADAYVOL2019 IS NULL And ABDAYVOL2050 IS NULL And BADAYVOL2050 IS NULL And sun_cloud_id LIKE '%,%'"
arcpy.Delete_management("destination_lyr")
arcpy.MakeFeatureLayer_management (fc, "destination_lyr", sql)

get_count("destination_lyr")


In [ ]:

ref_sql = 'DIR IN (-1, 1)'
arcpy.Delete_management("ref_lyr")
arcpy.MakeFeatureLayer_management ("tdm_project", "ref_lyr", ref_sql)

arcpy.analysis.Buffer('destination_lyr', 'destination_buffer', '400 Feet', "FULL", "FLAT", "NONE", None, "PLANAR")


# route = 'routes_split_intersection'
ref = 'ref_lyr'
buffer = 'destination_buffer'


fields = ['SHAPE@', 'OID@', 'route_id', 'suncloudid', 'flag']

ref_fields = ['SUNCLOUDID']

In [ ]:
dual_single = "dual_single_intersect"
# Run intersect
arcpy.analysis.Intersect(
    in_features=['destination_buffer', 'ref_lyr'],
    out_feature_class=dual_single ,
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)


In [ ]:
# In the intersect table
# for each orig_fid
# get all rows
# perform name check, functional class check
# leavel the two longest segsments
# sum each key field values


# create a reference fields
ref_fields = []
for f in key_fields:
    ref_fields.append('{}_1'.format(f))

ref_fields

In [ ]:
def func_match(functional_class, desc):
    # print(functional_class, desc)
    if (desc ==  'Freeway' and functional_class in [1,2]):
        result = True
    elif (desc ==  'Expressway' and functional_class in [2,3]):
        result = True
    elif (desc == '6 Legged Arterial' and functional_class in [2, 4]):
        result = True
    elif (desc in ['Arterial']  and functional_class in [2,3,4,5, 6]):
        result = True
    elif (desc in ['Collector/Frontage Rd', 'Major Collector', 'Minor Arterial']  and functional_class in [3,4,5, 6]):
        result = True
    elif (desc in ['Collector/Frontage Rd']  and functional_class in [3,4,5,6,7]):
        result = True
    elif (desc == 'CD Road' and functional_class in [2,3,4,5,6,7]):
        result = True
    elif (desc == 'Collector' and functional_class in [4,5]):
        result = True
    elif (desc ==  'Major Arterial' and functional_class in [3, 4]):
        result = True
    elif (desc == 'Principal Arterial' and functional_class in [3]):
        result = True
    elif (desc == 'Local' and functional_class in [7]):
        result = True
    else:
        result = False
    # print(result)
    return result

In [ ]:
# Get all the orig_fid
dual_single = "dual_single_intersect"
unique_oids  = unique_values(dual_single, 'ORIG_FID')
unique_oids


destination = 'destination_lyr'
fields = ['SUNCLOUDID', 'Shape_Length','functional_class', 'FUNCLASSDESC'] +ref_fields
print(fields)
route_fields = ['OID@', 'sun_cloud_id', 'ABLANES2019','BALANES2019','ABLANES2050','BALANES2050',
                'ABDAYVOL2019', 'BADAYVOL2019','ABDAYVOL2050','BADAYVOL2050', 
                'ABCAPACITY2019','BACAPACITY2019', 'ABCAPACITY2050', 'BACAPACITY2050']
for oid in unique_oids:
    
    if 1==1:
        print('processing {}...'.format(oid))
        ref = [row for row in arcpy.da.SearchCursor(in_table =dual_single, 
                                                        field_names =fields, 
                                                        where_clause="ORIG_FID ={}".format(oid),
                                                        
                                                        sql_clause= ('DISTINCT SUNCLOUDID', 'ORDER BY Shape_Length DESC'))]
        # print(ref)
        candidates = []
        for item in ref:
            # print(item[2])
            if(func_match(item[2], item[3])==True):
                candidates.append(item)
        if len(candidates)>1:
            candidates= candidates[:2]
            # print(candidates)
            _query = 'OBJECTID = {}'.format(oid)
            # print(_query)
            with arcpy.da.UpdateCursor(destination, route_fields, _query) as cursor:
                for row in cursor:
                    
                    c1 = candidates[0]
                    c2 = candidates[1]
                    # sc id
                    sc_ids = '{0},{1}'.format(c1[0],c2[0])
                    # print(sc_ids)
                    row[1]=sc_ids
                    # TODO: simplify this using index
                    row[2] = int(c1[4] or 0)+int(c2[4] or 0)
                    # BA LANES 2019
                    row[3] = int(c1[5] or 0)+int(c2[5] or 0)
                    #  ab lanes 2050
                    row[4] = int(c1[6] or 0)+int(c2[6] or 0)
                    #  ba lanes 2050
                    row[5] = int(c1[7] or 0)+int(c2[7] or 0)
                    # ab vol 2019
                    row[6] = int(c1[8] or 0)+int(c2[8] or 0)
                    # ba vol 2019
                    row[7] = int(c1[9] or 0)+int(c2[9] or 0)
                    # ab vol 2050
                    row[8] = int(c1[10] or 0)+int(c2[10] or 0)
                    # ba vol 2050
                    row[9] = int(c1[12] or 0)+int(c2[12] or 0)
                    # ab capacity 2019
                    row[10] = int(c1[13] or 0)+int(c2[13] or 0)
                    # ba capacity 2019
                    row[11] = int(c1[15] or 0)+int(c2[15] or 0)
                    # ab capacity 2050
                    row[12] = int(c1[16] or 0)+int(c2[16] or 0)
                    # ab capacity 2050
                    row[13] = int(c1[17] or 0)+int(c2[17] or 0)
                    cursor.updateRow(row)
        # if there's only one match, it's due to error in road representation type value. Categorize this as simple case and process later using the simple case code. 
        if len(candidates)==1:
            with arcpy.da.UpdateCursor(destination, ['conflation_type'], _query) as cursor:
                for row in cursor:
                    row[0]='wtf'
                    cursor.updateRow(row)

We changed conflation type to 'simple' where applicable.  Attribute those cases. 
- If there are sun_cloud_id with multiple values(dual to signle type), examine those in the map and update the sc ID where applicable. 

In [ ]:
fc = 'destination'
sql="conflation_type in ('simple')"

arcpy.MakeFeatureLayer_management(fc, 'simple_lyr', sql)
print(arcpy.GetCount_management('simple_lyr')[0])

In [ ]:
joined = arcpy.management.AddJoin(
    in_layer_or_view='simple_lyr',
    in_field="sun_cloud_id",
    join_table="tdm_project",
    join_field="SUNCLOUDID",
    index_join_fields="INDEX_JOIN_FIELDS"
)

In [ ]:
# get_count(joined)
# joinCheck(joined)

In [ ]:
# transfer over all the key field values
for k in key_fields:
    print(k)
    arcpy.management.CalculateField(
    in_table=joined,
    field="{0}.{1}".format(fc, k),
    expression="!tdm_project.{}!".format(k),
    expression_type="PYTHON3",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Calculate the total fields

In [ ]:
# Calculate the total fields.
sql = "sun_cloud_id <> 'no_source'"
selected = arcpy.SelectLayerByAttribute_management('destination', 'NEW_SELECTION', sql)
arcpy.management.CalculateField(
    in_table =selected,
    field="TOTDAYVOL2019",
    expression="int(!ABDAYVOL2019! or 0) + int(!BADAYVOL2019! or 0)",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
# Calculate the total fields.
sql = "sun_cloud_id <> 'no_source'"
selected = arcpy.SelectLayerByAttribute_management('destination', 'NEW_SELECTION', sql)
arcpy.management.CalculateField(
    in_table=selected,
    field="TOTDAYVOL2050",
    expression="int(!ABDAYVOL2050! or 0) + int(!BADAYVOL2050! or 0)",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

#### Process the HOV lanes.

In [ ]:
# select destination_restore where functional_class IN (1, 2)
# tdm_hov is the source

selected = arcpy.SelectLayerByAttribute_management('destination', "NEW_SELECTION", 'functional_class IN (1, 2)')
print(arcpy.GetCount_management(selected)[0])


possible_hov = arcpy.management.SelectLayerByLocation(
    in_layer=selected,
    overlap_type="WITHIN_A_DISTANCE",
    select_features="tdm_hov",
    search_distance="500 FeetInt",
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

print(arcpy.GetCount_management(possible_hov)[0])

# use the out-of-the-box transfer attribute tool with 500ft search distance

In [ ]:
arcpy.analysis.Buffer(
    in_features=possible_hov,
    out_feature_class="hov_buffer",
    buffer_distance_or_field="300 FeetInt",
    line_side="FULL",
    line_end_type="FLAT",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR"
)

Add hov fields.

In [ ]:
fc = 'destination'
hov_fields = [
['sun_cloud_id_hov', 'TEXT'],   
['ABLANES2019_hov', 'SHORT'],
['BALANES2019_hov','SHORT'],
['ABLANES2050_hov','SHORT'],
['BALANES2050_hov','SHORT'],
['ABDAYVOL2019_hov','DOUBLE'],
['BADAYVOL2019_hov','DOUBLE'],
['TOTDAYVOL2019_hov','DOUBLE'],
['ABDAYVOL2050_hov', 'DOUBLE'],
['BADAYVOL2050_hov','DOUBLE'],
['TOTDAYVOL2050_hov','DOUBLE'],
['ABCAPACITY2019_hov','DOUBLE'],
['BACAPACITY2019_hov','DOUBLE'],
['ABCAPACITY2050_hov','DOUBLE'],
['BACAPACITY2050_hov','DOUBLE']]

arcpy.management.AddFields(
    fc,hov_fields)
featureclass = fc
field_names = [f.name for f in arcpy.ListFields(featureclass)]
field_names

hov conflation function.

In [ ]:
# def tdm_conflation_hov(distance, in_fc, source_fc):

#     hov_query = 'hov = 1 And sun_cloud_id_hov IS NULL'
#     print(hov_query)

#     arcpy.Delete_management('destination_lyr')
#     arcpy.management.SelectLayerByAttribute( in_fc, 'CLEAR_SELECTION') 
#     arcpy.MakeFeatureLayer_management(in_fc, 'destination_lyr', hov_query)
#     get_count('destination_lyr')
    
#     # Create buffer around the routes with no SC ID. 
#     buffer_table = "hov_Buffer_{}".format(distance)
#     arcpy.analysis.Buffer(
#         in_features='destination_lyr',
#         out_feature_class=buffer_table,
#         buffer_distance_or_field="{} FeetInt".format(distance),
#         line_side="FULL",
#         line_end_type="FLAT",
#         dissolve_option="NONE",
#         dissolve_field=None,
#         method="PLANAR"
#     )

#     # # create a intersect table
#     intersect = "{}_hov_intersect".format(buffer_table)
#     arcpy.analysis.PairwiseIntersect(
#         in_features="{0};{1}".format(buffer_table, source_fc),
#         out_feature_class=intersect,
#         join_attributes="ALL",
#         cluster_tolerance=None,
#         output_type="INPUT"
#     )

#     org_fids = [row[0] for row in arcpy.da.SearchCursor(
#                 in_table=intersect, 
#                 field_names='ORIG_FID', 
#                 sql_clause= ('DISTINCT ORIG_FID', 'ORDER BY ORIG_FID ASC'))]
     
#     # For Each OID, leave the longest segment and delete the rest
#     fields = ['Shape_Length']
#     for oid in org_fids:
#         print('Processing oid = {}'.format(oid))
#         _query = 'ORIG_FID={}'.format(oid)
#         arcpy.MakeFeatureLayer_management(intersect, 'intersect_lyr')
#         sel = arcpy.SelectLayerByAttribute_management('intersect_lyr', 'NEW_SELECTION', _query)

#         # get_count(sel)

#         length = [row[0] for row in arcpy.da.SearchCursor(
#                 in_table=sel, 
#                 field_names='Shape_Length')]

#         max_len = max(length)
#         _query = 'ORIG_FID = {} and Shape_Length <{}'.format(oid, max_len)
#         sub_sel = arcpy.SelectLayerByAttribute_management(sel, 'SUBSET_SELECTION', _query)
#         if int(arcpy.GetCount_management(sub_sel)[0])>0:

#             arcpy.DeleteFeatures_management(sub_sel)

#     # join it back to the routes table and transfer the sc id. 
#     arcpy.Delete_management('null_layer')
#     arcpy.MakeFeatureLayer_management(in_fc, 'null_layer', 'sun_cloud_id_hov is null and hov=1')
#     arcpy.Delete_management('intersect_lyr')   
#     arcpy.MakeFeatureLayer_management(intersect, 'intersect_lyr')


#     joined = arcpy.management.AddJoin(
#         in_layer_or_view="null_layer",
#         in_field="OBJECTID",
#         join_table='intersect_lyr',
#         join_field="ORIG_FID",
#         join_type="",
#         index_join_fields="INDEX_JOIN_FIELDS"
#         )

#     arcpy.management.CalculateField(

#         in_table=joined,
#         field="{}.sun_cloud_id_hov".format(in_fc),
#         expression="""!{}.SUNCLOUDID_1!""".format(intersect),
#         expression_type="PYTHON3",
#         code_block="",
#         field_type="TEXT",
#         enforce_domains="NO_ENFORCE_DOMAINS"
#     )

#     # garbage collection
#     arcpy.Delete_management(joined)
# distances = [50, 100, 150, 200, 300]
# # for d in distances:
# #     tdm_conflation_hov(d, 'destination', 'tdm_hov')

Add up the hov values. 

In [ ]:
# where sun_cloud_id hov is not null, add up hov vlaues

sql_string = 'sun_cloud_id_hov is not null'
arcpy.Delete_management('hov_destination_lyr')
arcpy.MakeFeatureLayer_management('destination', 'hov_destination_lyr', sql_string)
get_count('hov_destination_lyr')

In [ ]:
hov_ids = unique_values('hov_destination_lyr' , 'sun_cloud_id_hov')
hov_ids

In [ ]:
hov_fields = [
 'ABDAYVOL2019',
 'BADAYVOL2019',
 'TOTDAYVOL2019',
 'ABDAYVOL2050',
 'BADAYVOL2050',
 'ABCAPACITY2019',
 'BACAPACITY2019',
 'ABCAPACITY2050',
 'BACAPACITY2050',
 'OID@'
]
for id in hov_ids:
    # if id=='MAG_17517':
    if 1==1:
        with arcpy.da.UpdateCursor('hov_destination_lyr', hov_fields, "sun_cloud_id_hov = '{}'".format(id)) as uCursor:

            lookup_sql = "SUNCLOUDID = '{}'".format(id)
            print(lookup_sql)
            values = [r for r in arcpy.da.SearchCursor('tdm_hov', hov_fields, lookup_sql)]
            print(values)
            
            for row in uCursor:
                print(row)
                i=0
                print('oid', row[9])
                while i <= 8:
                    val = values[0][i]
                    # if the hov value is greater than 0, add it to the destination value
                    if(val and val>0):
                        print(hov_fields[i])
                        print(row[i])
                        print(val)
                        row[i] = int(row[i] or 0) +val
                        print(row[i])
                    i = i + 1
                uCursor.updateRow(row)
        
    

### Field clean up.

Simplify fields.

In [ ]:
fc = 'destination'


arcpy.management.CalculateField(
    in_table=fc,
    field="volume_2019",
    expression="max(int(!ABDAYVOL2019! or 0), int(!BADAYVOL2019! or 0), int(!TOTDAYVOL2019! or 0))",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table=fc,
    field="volume_2050",
    expression="max(int(!ABDAYVOL2050! or 0), int(!BADAYVOL2050! or 0), int(!TOTDAYVOL2050! or 0))",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
fc='destination'
arcpy.management.CalculateField(
    in_table=fc,
    field="capacity_2019",
    expression="int(!ABCAPACITY2019! or 0)+int(!BACAPACITY2019! or 0)",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
fc='destination'
arcpy.management.CalculateField(
    in_table=fc,
    field="capacity_2050",
    expression="int(!ABCAPACITY2050! or 0)+int(!BACAPACITY2050! or 0)",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table=fc,
    field="lanes_2019",
    expression="lane_sum(!ABLANES2019!,!BALANES2019!)",
    expression_type="PYTHON3",
    code_block="""def lane_sum(a, b):
    
    if (a is None):
        a=0
    if (b is None):
        b=0
    result = a + b
    if (result == 0):
        return None
    else:
        return result""",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table=fc,
    field="lanes_2050",
    expression="lane_sum(!ABLANES2050!,!BALANES2050!)",
    expression_type="PYTHON3",
    code_block="""def lane_sum(a, b):
    
    if (a is None):
        a=0
    if (b is None):
        b=0
    result = a + b
    if (result == 0):
        return None
    else:
        return result""",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table='destination',
    field="vc_2019",
    expression="get_vc(!volume_2019!,!capacity_2019!)",
    expression_type="PYTHON3",
    code_block="""def get_vc(vol, cap):
    if cap>0:
        return round(vol/cap,3)
    
    """,
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table='destination',
    field="vc_2050",
    expression="get_vc(!volume_2050!,!capacity_2050!)",
    expression_type="PYTHON3",
    code_block="""def get_vc(vol, cap):
    if cap>0:
        return round(vol/cap,3)
    
    """,
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

Drop fields and unsplit. 


In [ ]:
fc = 'destination'

In [ ]:
# drop and rename fields
required_fields = ['OBJECTID', 'route_id', 'functional_class', 'sun_cloud_id',
'volume_2019','volume_2050', 'lanes_2019','lanes_2050','capacity_2019', 'capacity_2050',
'vc_2019', 'vc_2050']

In [ ]:
all_fields = arcpy.ListFields(fc)
f_names = [f.name for f in all_fields if f.required == False]

to_delete = [i for i in f_names if i not in required_fields]
to_delete

In [ ]:
arcpy.DeleteField_management(fc, to_delete)

In [ ]:
target_fields = [
    ('sun_cloud_id', 'sun_cloud_id', 'Segment ID'),
    ('volume_2019', 'volume_2019', 'Daily Volume 2019'),
    ('volume_2050','volume_2050','Daily Volume 2050'),
    ('lanes_2019','lanes_2019','Lanes 2019'),
    ('lanes_2050','lanes_2050','Lanes 2050'),
    ('capacity_2019','capacity_2019','Capacity 2019'),
    ('capacity_2050','capacity_2050','Capacity 2050'),
    ('vc_2019','vc_2019','Volume/Capacity Ratio 2019'),
    ('vc_2050','vc_2050','Volume/Capacity Ratio 2050')

]
for current, new, alias in target_fields:
    arcpy.management.AlterField(fc, current, new, alias)

Decode functional class.

In [ ]:
fc='destination'
arcpy.AddField_management(fc, 'functional_classification_code', 'SHORT')

In [ ]:
# copy over the functional class values
arcpy.management.CalculateField(
    in_table=fc,
    field="functional_classification_code",
    expression="!functional_class!",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.AddField_management(fc, 'functional_classification', 'TEXT')
arcpy.management.CalculateField(
    in_table=fc,
    field="functional_classification",
    expression="!functional_class!",
    expression_type="PYTHON3",

    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
fc_dict = {
    1 : 'Interstate',
    2 : 'Other Freeways and Expressways',
    3 : 'Other Principal Arterial',
    4 : 'Minor Arterial',
    5 : 'Major Collector',
    6 : 'Minor Collector',
    7:  'Local'
}
arcpy.management.CreateDomain(db, "functional class", 
                              "functional class", "TEXT", "CODED")

In [ ]:
for i in fc_dict:
    arcpy.AddCodedValueToDomain_management(db, 'functional class', i, fc_dict[i])

# assign domain to field
arcpy.management.AssignDomainToField(fc, 'functional_classification', 'functional class')

In [ ]:
joinCheck(fc)

In [ ]:
target_fields = [
    ('functional_classification', 'functional_classification', 'Functional Classification'),
    ('functional_classification_code', 'functional_classification_code', 'Functional Classification_code'),

]
for current, new, alias in target_fields:
    arcpy.management.AlterField(fc, current, new, alias)

arcpy.DeleteField_management(fc, 'functional_class')

Unsplit.

In [ ]:
# unsplit
arcpy.management.UnsplitLine(
    in_features="",
    out_feature_class="ExistingRoadways_UnsplitLine",
    dissolve_field="route_id;sun_cloud_id;volume_2019;volume_2050;lanes_2019;lanes_2050;capacity_2019;capacity_2050;vc_2019;vc_2050;functional_classification_code;functional_classification",
    statistics_fields=None,
    concatenation_separator=""
)